In [0]:
import pyspark
import json
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from datetime import datetime

In [0]:
common_event = StructType([StructField('trade_dt', DateType(), True),
                           StructField('rec_type', StringType(), True),
                           StructField('symbol', StringType(), True),
                           StructField('exchange', StringType(), True),
                           StructField('event_tm', TimestampType(), True),
                           StructField('event_seq_nb', IntegerType(), True),
                           StructField('arrival_tm', TimestampType(), True),
                           StructField('trade_pr', FloatType(), True),
                           StructField('bid_pr', FloatType(), True),
                           StructField('bid_size', IntegerType(), True),
                           StructField('ask_pr', FloatType(), True),
                           StructField('ask_size', IntegerType(), True),
                           StructField('partition', StringType(), True)])

In [0]:
def parse_csv(line):
    record_type_pos = 2
    record = line.split(",")

    try:
        if record[record_type_pos] == 'T':
              return (datetime.strptime(record[0], '%Y-%m-%d').date(), 
                      record[2], 
                      record[3], 
                      record[6], 
                      datetime.strptime(record[4], '%Y-%m-%d %H:%M:%S.%f'), 
                      int(record[5]), 
                      datetime.strptime(record[1], '%Y-%m-%d %H:%M:%S.%f'), 
                      float(record[7]), 
                      None, 
                      None, 
                      None, 
                      None, 
                      record[2])
        elif record[record_type_pos] == 'Q':
              return (datetime.strptime(record[0], '%Y-%m-%d').date(), 
                      record[2], 
                      record[3], 
                      record[6], 
                      datetime.strptime(record[4], '%Y-%m-%d %H:%M:%S.%f'), 
                      int(record[5]), 
                      datetime.strptime(record[1], '%Y-%m-%d %H:%M:%S.%f'), 
                      None, 
                      float(record[7]), 
                      int(record[8]), 
                      float(record[9]), 
                      int(record[10]), 
                      record[2])
    except:
        return (None, None, None, None, None, None, None, None, None, None, None, None, 'B')

In [0]:
spark = SparkSession.builder.getOrCreate()

In [0]:
spark.conf.set(
  "fs.azure.account.key.somestorage.blob.core.windows.net",
  "somekey")

raw_csv_05 = spark.sparkContext.textFile("wasbs://somecontainer@somestorage.blob.core.windows.net/csv/2020-08-05/NYSE")
parsed_csv_05 = raw_csv_05.map(lambda line: parse_csv(line))
csv_data_05 = spark.createDataFrame(parsed_csv_05, schema=common_event)

raw_csv_06 = spark.sparkContext.textFile("wasbs://somecontainer@somestorage.blob.core.windows.net/csv/2020-08-06/NYSE")
parsed_csv_06 = raw_csv_06.map(lambda line: parse_csv(line))
csv_data_06 = spark.createDataFrame(parsed_csv_06, schema=common_event)

csv_data = csv_data_05.union(csv_data_06)

In [0]:
csv_data.show()

+----------+--------+------+--------+--------------------+------------+-------------------+---------+---------+--------+---------+--------+---------+
 trade_dt|rec_type|symbol|exchange| event_tm|event_seq_nb| arrival_tm| trade_pr| bid_pr|bid_size| ask_pr|ask_size|partition|
+----------+--------+------+--------+--------------------+------------+-------------------+---------+---------+--------+---------+--------+---------+
2020-08-05| Q| SYMA| NYSE|2020-08-05 09:34:...| 1|2020-08-05 09:30:00| null| 75.30255| 100| 75.35917| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 09:40:...| 2|2020-08-05 09:30:00| null| 77.20875| 100| 78.90918| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 09:50:...| 3|2020-08-05 09:30:00| null| 77.15973| 100| 77.33205| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 09:57:...| 4|2020-08-05 09:30:00| null|79.299774| 100| 80.08399| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:06:...| 5|2020-08-05 09:30:00| null|77.863495| 100| 78.30821| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:11:...| 6|2020-08-05 09:30:00| null| 78.74799| 100|80.329056| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:17:...| 7|2020-08-05 09:30:00| null| 77.16837| 100| 77.97598| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:23:...| 8|2020-08-05 09:30:00| null| 78.06551| 100| 78.80516| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:31:...| 9|2020-08-05 09:30:00| null| 76.69653| 100| 78.56943| 100| Q|
2020-08-05| T| SYMA| NYSE|2020-08-05 10:37:...| 10|2020-08-05 09:30:00|79.194885| null| null| null| null| T|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:45:...| 11|2020-08-05 09:30:00| null| 75.10474| 100|77.096565| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:54:...| 12|2020-08-05 09:30:00| null| 74.52148| 100| 75.66917| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 11:03:...| 13|2020-08-05 09:30:00| null| 75.16876| 100| 76.70698| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 11:12:...| 14|2020-08-05 09:30:00| null| 74.56633| 100|74.621376| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 11:19:...| 15|2020-08-05 09:30:00| null| 74.49803| 100|76.475914| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 11:27:...| 16|2020-08-05 09:30:00| null| 75.22889| 100|76.143684| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 11:35:...| 17|2020-08-05 09:30:00| null| 76.62792| 100| 76.99149| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 11:42:...| 18|2020-08-05 09:30:00| null|77.094666| 100|78.159676| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 11:51:...| 19|2020-08-05 09:30:00| null|78.335365| 100| 79.39139| 100| Q|
2020-08-05| T| SYMA| NYSE|2020-08-05 11:56:...| 20|2020-08-05 09:30:00| 76.49093| null| null| null| null| T|
+----------+--------+------+--------+--------------------+------------+-------------------+---------+---------+--------+---------+--------+---------+
only showing top 20 rows

In [0]:
def parse_json(line):
    line = json.loads(line)
    record_type = line['event_type']

    try:
        if record_type == 'T':
              return (datetime.strptime(line['trade_dt'], '%Y-%m-%d').date(), 
                      record_type, 
                      line['symbol'], 
                      line['exchange'], 
                      datetime.strptime(line['event_tm'], '%Y-%m-%d %H:%M:%S.%f'), 
                      line['event_seq_nb'], 
                      datetime.strptime(line['file_tm'], '%Y-%m-%d %H:%M:%S.%f'), 
                      line['price'], 
                      None, 
                      None, 
                      None, 
                      None, 
                      record_type)
        elif record_type == 'Q':
              return (datetime.strptime(line['trade_dt'], '%Y-%m-%d').date(), 
                      record_type, 
                      line['symbol'], 
                      line['exchange'], 
                      datetime.strptime(line['event_tm'], '%Y-%m-%d %H:%M:%S.%f'), 
                      line['event_seq_nb'], 
                      datetime.strptime(line['file_tm'], '%Y-%m-%d %H:%M:%S.%f'), 
                      None, 
                      line['bid_pr'], 
                      line['bid_size'], 
                      line['ask_pr'], 
                      line['ask_size'], 
                      record_type)
    except:
        return (None, None, None, None, None, None, None, None, None, None, None, None, 'B')

In [0]:
raw_json_05 = spark.sparkContext.textFile("wasbs://somecontainer@somestorage.blob.core.windows.net/json/2020-08-05/NASDAQ")
parsed_json_05 = raw_json_05.map(lambda line: parse_json(line))
json_data_05 = spark.createDataFrame(parsed_json_05, schema=common_event)

raw_json_06 = spark.sparkContext.textFile("wasbs://somecontainer@somestorage.blob.core.windows.net/json/2020-08-06/NASDAQ")
parsed_json_06 = raw_json_06.map(lambda line: parse_json(line))
json_data_06 = spark.createDataFrame(parsed_json_06, schema=common_event)

json_data = json_data_05.union(json_data_06)

In [0]:
json_data.show()

+----------+--------+------+--------+--------------------+------------+-------------------+---------+---------+--------+---------+--------+---------+
 trade_dt|rec_type|symbol|exchange| event_tm|event_seq_nb| arrival_tm| trade_pr| bid_pr|bid_size| ask_pr|ask_size|partition|
+----------+--------+------+--------+--------------------+------------+-------------------+---------+---------+--------+---------+--------+---------+
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 09:36:...| 1|2020-08-05 09:30:00| null| 76.10017| 100| 77.9648| 100| Q|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 09:42:...| 2|2020-08-05 09:30:00| null| 75.44373| 100| 75.94453| 100| Q|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 09:48:...| 3|2020-08-05 09:30:00| null|78.847984| 100| 80.69115| 100| Q|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 09:53:...| 4|2020-08-05 09:30:00| null| 74.98337| 100| 76.16257| 100| Q|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 10:00:...| 5|2020-08-05 09:30:00| null| 76.71408| 100|77.855125| 100| Q|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 10:05:...| 6|2020-08-05 09:30:00| null| 76.78901| 100| 77.79983| 100| Q|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 10:13:...| 7|2020-08-05 09:30:00| null|76.370995| 100| 76.94632| 100| Q|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 10:21:...| 8|2020-08-05 09:30:00| null| 77.32601| 100| 77.65018| 100| Q|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 10:30:...| 9|2020-08-05 09:30:00| null| 77.11395| 100| 77.52986| 100| Q|
2020-08-05| T| SYMA| NASDAQ|2020-08-05 10:38:...| 10|2020-08-05 09:30:00| 77.7757| null| null| null| null| T|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 10:45:...| 11|2020-08-05 09:30:00| null| 77.69893| 100| 79.18605| 100| Q|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 10:54:...| 12|2020-08-05 09:30:00| null| 75.09529| 100| 75.83132| 100| Q|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 11:02:...| 13|2020-08-05 09:30:00| null| 77.76547| 100| 79.41345| 100| Q|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 11:11:...| 14|2020-08-05 09:30:00| null| 76.39994| 100| 76.83733| 100| Q|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 11:19:...| 15|2020-08-05 09:30:00| null| 79.09882| 100| 79.83007| 100| Q|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 11:29:...| 16|2020-08-05 09:30:00| null| 74.72294| 100| 76.05085| 100| Q|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 11:34:...| 17|2020-08-05 09:30:00| null| 78.06235| 100| 78.36705| 100| Q|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 11:41:...| 18|2020-08-05 09:30:00| null|78.716835| 100| 80.4561| 100| Q|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 11:50:...| 19|2020-08-05 09:30:00| null| 79.1717| 100| 80.14798| 100| Q|
2020-08-05| T| SYMA| NASDAQ|2020-08-05 11:58:...| 20|2020-08-05 09:30:00|75.715225| null| null| null| null| T|
+----------+--------+------+--------+--------------------+------------+-------------------+---------+---------+--------+---------+--------+---------+
only showing top 20 rows

In [0]:
data = csv_data.union(json_data)

Out[21]: 1200

In [0]:
data.printSchema()

root
-- trade_dt: date (nullable = true)
-- rec_type: string (nullable = true)
-- symbol: string (nullable = true)
-- exchange: string (nullable = true)
-- event_tm: timestamp (nullable = true)
-- event_seq_nb: integer (nullable = true)
-- arrival_tm: timestamp (nullable = true)
-- trade_pr: float (nullable = true)
-- bid_pr: float (nullable = true)
-- bid_size: integer (nullable = true)
-- ask_pr: float (nullable = true)
-- ask_size: integer (nullable = true)
-- partition: string (nullable = true)

In [0]:
data.write.partitionBy("partition").mode("overwrite").parquet("wasbs://somecontainer@somestorage.blob.core.windows.net/output_dir")